In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [24]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [6]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [7]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [34]:
ADD_CONST = 1e-5
CENTER = -2.5
SCALE = 5
XFORM = True

In [35]:
def xform(a):
    if XFORM:
        return((np.log(a+ADD_CONST) - CENTER) / SCALE)
    else:
        return(a)

In [36]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xform(xbatch), ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [37]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [38]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [115]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, input_shape=input_shape)) # 64
#model.add(keras.layers.Conv2D(50, 1, input_shape=input_shape))
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0))
#model.add(keras.layers.MaxPooling2D(2,1))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.SpatialDropout2D(.1))
model.add(keras.layers.Conv2D(32, 1))  # 32
#model.add(keras.layers.Conv2D(20, 1))  # 32
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.SpatialDropout2D(.2))
#model.add(keras.layers.Conv2D(16, 1, activation='relu'))
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(40, (2,4))) # 16
model.add(keras.layers.Conv2D(16, (2,4))) # 16
#model.add(keras.layers.Conv2D(16, (2,6))) # 16
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.SpatialDropout2D(.3))
model.add(keras.layers.Conv2D(25, (4,6))) # 29,5
#model.add(keras.layers.Conv2D(32, (4,6))) # 29,5
#model.add(keras.layers.Conv2D(25, 5)) # 29,5
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0.0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.SpatialDropout2D(.4))
#model.add(keras.layers.Conv2D(3, (2,5))) # 4,3
#model.add(keras.layers.Conv2D(4, (2,4))) # 4,3
model.add(keras.layers.Conv2D(3, (2,4))) # 4,3
#model.add(keras.layers.Conv2D(3, (2,3))) # 4,3
#model.add(keras.layers.Conv2D(4, 3)) # 4,3
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.MaxPooling2D(1,2))
#model.add(keras.layers.PReLU())
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(3, 1))
#model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dropout(.2)) # or .2
#model.add(keras.layers.Dropout(.25)) # or .2
#model.add(keras.layers.Dense(35, activation='relu'))
#model.add(keras.layers.Dense(25, activation='relu'))
model.add(keras.layers.Dropout(.6)) # .5 or .6
#model.add(keras.layers.GaussianDropout(.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [116]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=.003,decay=2e-4), 
              metrics=['accuracy'])

In [117]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_101 (Conv2D)          (None, 16, 16, 64)        8256      
_________________________________________________________________
leaky_re_lu_101 (LeakyReLU)  (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization_56 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 16, 16, 32)        2080      
_________________________________________________________________
leaky_re_lu_102 (LeakyReLU)  (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_57 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 15, 13, 16)        4112      
__________

In [118]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=60, 
                    steps_per_epoch=64)

Epoch 1/60
64/64 [==============================] - 3s 48ms/step - loss: 0.7021 - acc: 0.6777 - val_loss: 0.5722 - val_acc: 0.7651
Epoch 2/60
64/64 [==============================] - 1s 20ms/step - loss: 0.5961 - acc: 0.7524 - val_loss: 0.5843 - val_acc: 0.7573
Epoch 3/60
64/64 [==============================] - 1s 21ms/step - loss: 0.5795 - acc: 0.7632 - val_loss: 0.5614 - val_acc: 0.7598
Epoch 4/60
64/64 [==============================] - 1s 20ms/step - loss: 0.5781 - acc: 0.7666 - val_loss: 0.5626 - val_acc: 0.7681
Epoch 5/60
64/64 [==============================] - 1s 20ms/step - loss: 0.5633 - acc: 0.7710 - val_loss: 0.5759 - val_acc: 0.7646
Epoch 6/60
64/64 [==============================] - 1s 21ms/step - loss: 0.5740 - acc: 0.7500 - val_loss: 0.5385 - val_acc: 0.7778
Epoch 7/60
64/64 [==============================] - 1s 20ms/step - loss: 0.5298 - acc: 0.7729 - val_loss: 0.5300 - val_acc: 0.7598
Epoch 8/60
64/64 [==============================] - 1s 21ms/step - loss: 0.5177 - a

In [119]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(xform(c), axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

BigJimMP 82
Predicted:  0.8774294    Actual:  True 

BigJimMP 167
Predicted:  0.8721071    Actual:  True 

BloodyMerryMorning 73
Predicted:  0.0028755136    Actual:  False 

HangingOnTheTelephone 101
Predicted:  0.0010437892    Actual:  False 

BigJimMP 161
Predicted:  0.8865248    Actual:  True 

ImLookingThruYou 43
Predicted:  0.03490556    Actual:  False 

WalkThisWay1 49
Predicted:  0.001167191    Actual:  False 

09 Suddenly I See 44
Predicted:  0.00021952346    Actual:  False 

IThinkWereAloneNow2MP 53
Predicted:  7.9605576e-05    Actual:  False 

WalkThisWay1 48
Predicted:  0.00061431504    Actual:  False 



<code>
preproc sp d/o replaces b/n for middle layer (d/o=30/60) -> 2258, **2308**, 2236, 2585, 2487, 
    2599, 2238, **2326**
above w d/o=30/50 -> 2509
30/70 -> 2442
preproc sp d/o no b/n -> 2673, 5457
preproc, d/o=.6 -> 3027
preproc, d/o=.9 -> 2952
preproc, d/o=.7 -> 2951
with preproc -> 2936, 3173, 2644
baseline (from below):
*only 3f on last conv -> **2693**, 2891, 2753, 2821, 2613, 2613, *2672*, 2657, 2913, 
                         2588, 2878, 2883, 2580, 2597, 2523, **2703**, 2619, 2906, 2898,
                         *2745*, 3123, 2536 *
</code>    

<code>
last conv=(3,2x3) -> *2756*, 2559, 2826, *2715*, 2581, 2775, 2669, 2922, **2728**, 
                     2509, 2655, 2839, 2413, 2768, 2702, 2890, 3013
last conv=(3,3x4) -> 2823, 2784, 2746, 2921
last conv=(3,2x5) -> 2483, 3242, 2953
64-32-64-25-4 -> 3010
2x6, 4x6, 2x4 -> 3085
50-20-40-25-4 -> 2686, 2796, 3010, 3022
50-20-10-25-4 -> 2842, 2949
80-40-20-32-4 -> 3008
*only 3f on last conv -> *2693*, 2891, 2753, 2821, 2613, 2613, 2672, 2657, 2913, 
                         2588, 2878, 2883, 2580, 2597, 2523, **2703**, 2619, 2906, 2898,
                         *2745*, 3123 *
5f on last conv + (3, 1x1) -> 3455
add conv(2, 1x1) on top -> 2613, 2660, 3053, 2754
maxpool(1,2) at the top -> 2765, 2963
extra dense layer (35 d/o=.25) -> 3111
   
2x4,4x6,2x4,lr=003,decay=0002 baseline, from below:
2531, 2818-, 2422, 2825, 2613, 2576, 2871, 2619, 3204, 2818+, *2785*,
3081, 3235, 2715, 2605, *2761*, **2771**
</code>

<code>from 5f baseline:
lr=.004,decay=2e-4 -> 2659, 2679, 2934
lr=.002,decay=2e-4 -> 2932
lr=.003,decay=1e-4 -> 2835
lr=.003,decay=3e-4 -> 2968
lr=.002,decay=3e-4 -> 2879, 2977
lr=.004,decay=1e-4 -> 3127
lr=.004,decay=5e-4 -> 2696, 3148, 2900
*lr=.003,decay=2e-4 -> 2531, 2818, 2422, 2825, **2613**, 2576, 2871 *
lr=.003,decay=3e-5 -> 3129
extra dense layer (25 d/o=.3) -> 2786, **2873**, 3130, 3012, 2744
extra dense layer (50 d/o=.2) -> 3202, 2823, **2833**, 2776, 3153
change 32 conv to (2,1) -> 2893, 3380
add maxpool(2,1) on bottom -> 3599
add maxpool(2,1) on top -> 2857, 2977, 3170
add 2, 2x3 on top -> 3197, 3070
    
from 5f baseline:
drop bottom b/n -> 3050, 3032, 2667, 3353, 2736, 3132, 3210, 3199, 3359
drop top b/n -> 3266, 2870, 3306, 3039, 2917, 3201, 3040, 3304
60-25-50-20-5 -> 3237, 3136, 2531, 2818, 2422, 2825, 2613, 2576, 2871

80-50-20-32-5 -> 2939, 3086, 2849, 3110, 2990, 3200, 3111
50-20-14-25-4 -> 3379
80-50-16-25-4 -> 3165, 3079, 2868, 2994, 3093, 2995, 2951, 3268, 3145

add 2, 1x1 on top -> 3049, 2936, 2849
2x4, 4x7, 2x4 -> 2942, 2910
2x4, 4x5, 2x4 -> 3072, 3360from 5f baseline:
drop bottom b/n -> 3050, 3032, 2667, 3353, 2736, 3132, 3210, 3199, 3359
drop top b/n -> 3266, 2870, 3306, 3039, 2917, 3201, 3040, 3304
60-25-50-20-5 -> 3237, 3136

2x4, 4x6, 3x4 -> 2886, 2696, 2987, 3009, 2789, 3427, **2979**
2x4, 4x6, 2x3 -> 2613, 3383, 2632, 3046, *2889*, **2940**, 2666, 3039, *3019*
above w d/o=.65 -> 3238
2x4, 4x6, 2x6 -> 2849, **3003**, 2588, 3200, 3008
2x4, 4x8, 2x4 -> 2712, **2959**, 3199, 2987, 2934
2x6, 4x6, 2x4 -> 2893, *2861*, **2868**, 3101, *2821*, 2581, 3021, 3155, *2880*
    
*baseline 2x4, 4x6, 2x4 -> 2728, 2940, 3030, **2821**, 2440, 2911, 2914, 2924, 3134, 2708, 2534,
                           2910, 3352, 2870, 2790, *2816*, 2735, 2763, 2767, 2942, 3030,
                           2786, 2942, 2772, 3089, 2777, 2854, 2690, 2912, 2659, 2799,
                           2770, 2801, 3021, 2852, 2725, **2825**, 3249, 2635, 3211 *
    
Alternatives:
2x4, 5x5, 3x3      -> 2809, 3087, 3104, 2887, 2767, 2852, 2763, 3067, 2994, 3153, 3011,
                      **2929**, 2777, 2745, 2806, 3070, 3197
2x4, 4x6, 3x3      -> 3163, 2705, 2931, **2829**, 2915, 3111, 2765, 2791, 3152, 2813, 2797,
                      2771, *2836*, 2876, 3039, 2951, 2726, 3144, 2734, 2699, 2732,
                      2968, 2755, 3082, 2881, 2723, 2809, 3320, 2659, 2936, *2814*
</code>